# Chapter 2 Hydrology: Channel Geometry

1. [Introduction](#1.-Introduction)
2. [Simulation](#2.-Simulation)
3. [Self-Assessment](#3.-Self-Assessment)
    3.1 [Self-Assessment_1](#3.1-Self-Assessment_1)
    3.2 [Self-Assessment_2](#3.2-Self-Assessment_2)
    3.3 [Self-Assessment_3](#3.3-Self-Assessment_3)

### Channel Geometry Estimation: Theory, Applicability & Comparison

This module describes the theoretical foundations and practical considerations of three widely used methods for estimating stream channel geometry based on discharge and slope.

---

### 1. Leopold & Maddock Hydraulic Geometry

This method uses empirical power-law relationships derived from field observations:

$$
w = a_w \cdot Q^{b_w}
\quad ; \quad
d = a_d \cdot Q^{b_d}
\quad ; \quad
A = w \cdot d
$$

Where:
- $( Q $): discharge (m³/s)  
- $( w $): channel width (m)  
- $( d $): channel depth (m)  
- $( A $): cross-sectional area (m²)  
- $( a, b $): region-specific coefficients

###  Applicability
- Quick estimation for ungauged basins  
- Useful for regional planning and screening  
- Adaptable to different climate zones (temperate, arid, humid)

###  Limitations
- Generalized; not site-specific  
- Does not account for slope, roughness, or sediment load  
- Assumes stable channel form

---

### USGS Regional Regression Equations

Based on statistical regression of measured channel dimensions across multiple watersheds:

$$
w = a \cdot Q^{b}
\quad ; \quad
d = c \cdot Q^{d}
$$

Coefficients vary by region and are derived from field data (e.g., Bent & Waite, 2013).

###  Applicability
- Calibrated to real-world measurements  
- Useful for design in regions with published regression models  
- Can be integrated into hydrologic models

### Limitations
- Region-specific; not transferable  
- May not reflect future or altered hydrology  
- Limited to the range of data used in regression

---

### Manning Slope-Area Method

Based on open channel hydraulics and energy balance:

$$
Q = \frac{1}{n} \cdot A \cdot R^{2/3} \cdot S^{1/2}
$$

Where:
- $( Q $): discharge (m³/s)  
- $( A $): cross-sectional area (m²)  
- $( R $): hydraulic radius = $( A / P $)  
- $( P $): wetted perimeter (m)  
- $( S $): slope (m/m)  
- $( n $): Manning’s roughness coefficient

### Applicability
- Physics-based; accounts for slope and roughness  
- Useful for design, flood modeling, and sediment transport  
- Can be adapted to any channel shape

### Limitations
- Requires assumptions about channel geometry  
- Sensitive to roughness coefficient $( n $)  
- May not converge for extreme discharges or flat slopes

---

## 🔍 Comparative Summary

| Method                 | Basis         | Strengths                         | Limitations                        |
|------------------------|---------------|-----------------------------------|------------------------------------|
| Leopold & Maddock      | Empirical     | Fast, regional, scalable          | No slope or roughness input        |
| USGS Regression        | Statistical   | Calibrated to field data          | Region-specific, static            |
| Manning Slope-Area     | Hydraulic     | Physics-based, flexible           | Requires geometry assumptions      |

---

### Opportunities for Integration

- Combine empirical and hydraulic methods for cross-validation  
- Use regression models to initialize Manning parameters  
- Extend to sediment transport, stream power, and channel evolution  
- Apply in restoration, floodplain mapping, and climate adaptation

---

> “Channel geometry is shaped by the interplay of flow, slope, sediment, and landscape. These models offer complementary tools for understanding and designing resilient river systems.”

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

# 📐 Stream power
def stream_power(Q, S, rho=1000, g=9.81):
    return rho * g * Q * S  # Watts

# 📐 Method 1: Leopold & Maddock hydraulic geometry
def leopold_geometry(Q, region='temperate'):
    if region == 'temperate':
        a_w, b_w = 5.0, 0.5
        a_d, b_d = 0.3, 0.4
    elif region == 'arid':
        a_w, b_w = 7.0, 0.45
        a_d, b_d = 0.2, 0.35
    else:  # humid
        a_w, b_w = 6.0, 0.48
        a_d, b_d = 0.25, 0.38
    w = a_w * Q**b_w
    d = a_d * Q**b_d
    A = w * d
    return w, d, A

# 📐 Method 2: USGS regression (simplified form)
def usgs_geometry(Q):
    # Based on Bent & Waite (2013) for Northeast US
    w = 4.5 * Q**0.5
    d = 0.25 * Q**0.4
    A = w * d
    return w, d, A

# 📐 Method 3: Manning-based slope-area method
# 📐 Method 3: Manning-based slope-area method (revised)
def manning_geometry(Q, S, n=0.035):
    best_d, min_err = None, float('inf')
    for d in np.linspace(0.1, 20, 2000):  # Expanded depth range
        w = 5 + 2 * d  # trapezoidal: bottom width + side slopes
        A = w * d
        P = w + 2 * np.sqrt(1**2 + d**2)  # wetted perimeter
        R = A / P
        V = (1/n) * R**(2/3) * S**0.5
        Q_calc = V * A
        err = abs(Q_calc - Q)
        if err < min_err:
            min_err = err
            best_d = d
            best_w = w
            best_A = A
        if err < 1.0:  # relaxed tolerance
            return w, d, A

    print(f"⚠️ Manning method did not converge exactly. Best estimate within ±{min_err:.2f} m³/s.")
    return best_w, best_d, best_A

# 📊 Comparison plot
def compare_geometry(Q, S, region):
    omega = stream_power(Q, S)
    w1, d1, A1 = leopold_geometry(Q, region)
    w2, d2, A2 = usgs_geometry(Q)
    w3, d3, A3 = manning_geometry(Q, S)

    methods = ['Leopold & Maddock', 'USGS Regression', 'Manning Slope-Area']
    widths = [w1, w2, w3]
    depths = [d1, d2, d3]
    areas = [A1, A2, A3]

    # Output summary
    print(f"🌊 Discharge: {Q:.2f} m³/s | Slope: {S:.4f} m/m | Region: {region}")
    print(f"⚡ Stream Power: {omega:.2f} Watts\n")
    for i, m in enumerate(methods):
        print(f"🔹 {m}: Width = {widths[i]:.2f} m | Depth = {depths[i]:.2f} m | Area = {areas[i]:.2f} m²")

    # 📈 Plot comparison
    fig, ax = plt.subplots(figsize=(10, 5))
    for i, m in enumerate(methods):
        ax.plot([0, widths[i]], [0, 0], label=f"{m}", linewidth=2)
        ax.fill_between([0, widths[i]], 0, depths[i], alpha=0.3)
        ax.text(widths[i]/2, depths[i]/2, f"{m}", ha='center', va='center', fontsize=10)
    ax.set_title("Channel Geometry Comparison by Method")
    ax.set_xlabel("Width (m)")
    ax.set_ylabel("Depth (m)")
    ax.grid(True, linestyle='--', alpha=0.5)
    ax.legend()
    plt.tight_layout()
    plt.show()

# 🎛️ Interactive controls
interact(
    compare_geometry,
    Q=FloatSlider(value=20, min=1, max=500, step=5, description="Discharge Q (m³/s)"),
    S=FloatSlider(value=0.005, min=0.0001, max=0.05, step=0.0005, description="Slope S"),
    region=Dropdown(options=['temperate', 'arid', 'humid'], value='temperate', description="Region")
)

interactive(children=(FloatSlider(value=20.0, description='Discharge Q (m³/s)', max=500.0, min=1.0, step=5.0),…

<function __main__.compare_geometry(Q, S, region)>

## 3. Self-Assessment

### Leopold & Maddock Hydraulic Geometry: Quiz, Conceptual & Reflective Questions

This module reinforces understanding of hydraulic geometry theory and its application in stream channel analysis. It supports learning through multiple-choice questions, conceptual prompts, and reflective challenges.

---

### Conceptual Questions

1. **What does the exponent \( b_w \) in the equation \( w = a_w \cdot Q^{b_w} \) represent?**
   - A. The slope of the channel  
   - B. The rate at which width increases with discharge  
   - C. The roughness coefficient  
   - D. The initial channel width

2. **Which of the following best describes the relationship between discharge and channel depth in hydraulic geometry?**
   - A. Depth increases linearly with discharge  
   - B. Depth decreases as discharge increases  
   - C. Depth increases sub-linearly with discharge  
   - D. Depth remains constant

3. **What assumption underlies the Leopold & Maddock hydraulic geometry method?**
   - A. Channels are engineered and uniform  
   - B. Channels are in dynamic equilibrium with flow  
   - C. Sediment transport is negligible  
   - D. Flow velocity is constant across all discharges

4. **Which parameter is not explicitly included in the hydraulic geometry equations?**
   - A. Discharge  
   - B. Channel slope  
   - C. Channel width  
   - D. Flow velocity

5. **If a stream has a high \( b_w \) exponent, what does that imply?**
   - A. Width changes very little with discharge  
   - B. Width increases rapidly with discharge  
   - C. Depth dominates channel adjustment  
   - D. Velocity remains constant

---

### Code Interpretation Prompts

1. Why are width and depth modeled as power-law functions of discharge?  
2. How does the product \( w \cdot d \cdot v \) ensure consistency with the continuity equation \( Q = A \cdot v \)?  
3. What does it mean physically when width increases faster than depth?  
4. How would you calibrate the coefficients \( a \) and \( b \) for a specific watershed?  
5. Why might hydraulic geometry be less accurate in urban or engineered channels?

---

### Reflective Questions

1. **How does hydraulic geometry reflect the self-adjusting nature of rivers?**  
2. **Why is it useful to have empirical relationships for width and depth when designing restoration projects?**  
3. **What are the risks of applying hydraulic geometry equations without local calibration?**  
4. **How might climate change or land use shifts affect the validity of hydraulic geometry assumptions?**  
5. **In what ways could hydraulic geometry be integrated with sediment transport or stream power models?**

---

### Design Insight

> “Hydraulic geometry reveals how rivers scale their form to match flow. Understanding these relationships helps engineers and scientists design with nature, not against it.”

---